# Selección de características para análisis de sentimiento

Vamos a explorar ideas de la sesión 2 para la selección de características en el problema de análisis de sentimiento

## Entrenamiento y evaluación

Usamos una función para encapsular el entrenamiento y evaluación. 
 * Por simplificar, asumimos alguna de las cosas que serían mejorables - no usar las características de tests en la selección y entrenamiento
 * Experimentamos con diferentes procedimientos para seleccionar características. El clasificador recibe la función `featx` como parámetro 

In [ ]:
import collections
import nltk.metrics
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import movie_reviews
 

def evaluate_classifier(featx):
    negids = movie_reviews.fileids('neg')
    posids = movie_reviews.fileids('pos')
 
    negfeats = [(featx(movie_reviews.words(fileids=[f])), 'neg') for f in negids]
    posfeats = [(featx(movie_reviews.words(fileids=[f])), 'pos') for f in posids]
 
    negcutoff = len(negfeats)*3/4
    poscutoff = len(posfeats)*3/4
 
    trainfeats = negfeats[:negcutoff] + posfeats[:poscutoff]
    testfeats = negfeats[negcutoff:] + posfeats[poscutoff:]
    print 'train on %d instances, test on %d instances' % (len(trainfeats), len(testfeats))
 
    classifier = NaiveBayesClassifier.train(trainfeats)
    refsets = collections.defaultdict(set)
    testsets = collections.defaultdict(set)
 
    for i, (feats, label) in enumerate(testfeats):
        refsets[label].add(i)
        observed = classifier.classify(feats)
        testsets[observed].add(i)
 
    print 'accuracy:', nltk.classify.util.accuracy(classifier, testfeats)
    print 'pos precision:', nltk.metrics.precision(refsets['pos'], testsets['pos'])
    print 'pos recall:', nltk.metrics.recall(refsets['pos'], testsets['pos'])
    print 'pos F-measure:', nltk.metrics.f_measure(refsets['pos'], testsets['pos'])
    print 'neg precision:', nltk.metrics.precision(refsets['neg'], testsets['neg'])
    print 'neg recall:', nltk.metrics.recall(refsets['neg'], testsets['neg'])
    print 'neg F-measure:', nltk.metrics.f_measure(refsets['neg'], testsets['neg'])
    classifier.show_most_informative_features()


## Bag of words - all words

In [ ]:
def word_feats(words):
    return dict([(word, True) for word in words])
 
evaluate_classifier(word_feats)

## Filtrado de stopwords

In [ ]:
from nltk.corpus import stopwords
stopset = set(stopwords.words('english'))
 
def stopword_filtered_word_feats(words):
    return dict([(word, True) for word in words if word not in stopset])
 
evaluate_classifier(stopword_filtered_word_feats)

## Filtrando a N palabras más frecuentes

In [ ]:
max_types = 100

all_words = nltk.FreqDist(w.lower() for w in movie_reviews.words())
most_common = set([w for (w,f) in all_words.most_common(max_types)])

def most_common_word_feats(words):
    return dict([(word, True) for word in words 
                 if word in most_common and word not in stopset])

evaluate_classifier(most_common_word_feats)

## Filtrando por frecuencia

In [ ]:
minfreq = 10

most_frequent = [w for w in set(movie_reviews.words()) if all_words[w] > minfreq]

def most_frequent_word_feats(words):
    return dict([(word, True) for word in words if word in most_frequent and word not in stopset])

evaluate_classifier(most_common_word_feats)

## Bigramas

In [ ]:
b = nltk.bigrams(["Hola", "que", "tal", "estas", "?"])

[w for w in b ]

In [ ]:
import itertools

def bigram_feats(words):
    bigrams = nltk.bigrams(words)
    return dict([(word, True) for word in bigrams])

evaluate_classifier(bigram_feats)

## Bigramas y palabras

In [ ]:
import itertools

def word_and_bigram_feats(words):
    bigrams = nltk.bigrams(words)
    return dict([(word, True) for word in itertools.chain(words,bigrams)])

evaluate_classifier(word_and_bigram_feats)

## Seleccion de los bigramas más significativos 

In [ ]:
BigramCollocationFinder?

In [ ]:
BigramAssocMeasures.chi_sq?

In [ ]:
from nltk.collocations import BigramCollocationFinder
from nltk.metrics import BigramAssocMeasures
 
def bigram_word_feats(words, score_fn=BigramAssocMeasures.chi_sq, n=200):
    bigram_finder = BigramCollocationFinder.from_words(words)
    bigrams = bigram_finder.nbest(score_fn, n)
    return dict([(ngram, True) for ngram in itertools.chain(words, bigrams)])
 
evaluate_classifier(bigram_word_feats)


## Selección de las palabras más significativas

In [ ]:
from nltk.collocations import BigramCollocationFinder
from nltk.metrics import BigramAssocMeasures
from nltk.probability import FreqDist, ConditionalFreqDist
 

word_fd = FreqDist()
label_word_fd = ConditionalFreqDist()

for word in movie_reviews.words(categories = ['pos']):
    word_fd[word.lower()] += 1
    label_word_fd['pos'][word.lower()] +=1
    
for word in movie_reviews.words(categories = ['neg']):
    word_fd[word.lower()] += 1
    label_word_fd['neg'][word.lower()] += 1

pos_word_count = label_word_fd['pos'].N()
neg_word_count = label_word_fd['neg'].N()
total_word_count = pos_word_count + neg_word_count

word_scores = {}

for word, freq in word_fd.iteritems():
    pos_score = BigramAssocMeasures.chi_sq(label_word_fd['pos'][word],
        (freq, pos_word_count), total_word_count)
    neg_score = BigramAssocMeasures.chi_sq(label_word_fd['neg'][word],
        (freq, neg_word_count), total_word_count)
    word_scores[word] = pos_score + neg_score

best = sorted(word_scores.iteritems(), key=lambda (w,s): s, reverse=True)[:10000]
bestwords = set([w for w, s in best])    
    
def best_word_feats(words):
    return dict([(word, True) for word in words if word in bestwords])

print 'evaluating best word features'
evaluate_classifier(best_word_feats)
 

## Selección de los bigramas y las palabras más significativos

In [ ]:
def best_bigram_word_feats(words, score_fn=BigramAssocMeasures.chi_sq, n=200):
    bigram_finder = BigramCollocationFinder.from_words(words)
    bigrams = bigram_finder.nbest(score_fn, n)
    d = dict([(bigram, True) for bigram in bigrams])
    d.update(best_word_feats(words))
    return d
 
print 'evaluating best words + bigram chi_sq word features'
evaluate_classifier(best_bigram_word_feats)